Copyright 2023 Google LLC.
SPDX-License-Identifier: Apache-2.0

# Setup

Running on hosted runtimes currently not fully supported. The relevant code will be skipped unless you set `RUNNING_ON_HOSTED_RUNTIME = True`. If `RUNNING_ON_HOSTED_RUNTIME == False`, it's best to just skip this heading and got straight to **Imports**, as the `%%writefile` line magic isn't available in all environments.

## Running on hosted runtime

In [ ]:
#@title Set `RUNNING_ON_HOSTED_RUNTIME`
RUNNING_ON_HOSTED_RUNTIME = False #@param {type:"boolean"}

In [ ]:
#@title Write requirements.txt file
%%writefile requirements.txt
jupyter
pandas
tqdm
xgboost
opencv-python
matplotlib

In [ ]:
#@title Install requirements
if RUNNING_ON_HOSTED_RUNTIME:
  %pip install -r requirements.txt
  # Used to install GDAL
  !sudo apt-get install gdal-bin
  !sudo apt-get install libgdal-dev
  !export CPLUS_INCLUDE_PATH=/usr/include/gdal
  !export C_INCLUDE_PATH=/usr/include/gdal
  !pip install GDAL==$(gdal-config --version) --global-option=build_ext --global- option="-I/usr/include/gdal"

In [ ]:
#@title Write partition.py
%%writefile partition.py
"""
Module for helper functions for manipulating data and datasets.
"""

from dataclasses import dataclass
import pandas as pd

@dataclass
class PartitionedDataset:
  train: pd.DataFrame
  test: pd.DataFrame
  validation: pd.DataFrame

def partition(df) -> PartitionedDataset:
  train = df[df["lon"] < -55]
  test = df[(df["lon"] >= -55) & (df["lat"] > -2.85)]
  validation = df[(df["lon"] >= -55) & (df["lat"] <= -2.85)]
  return PartitionedDataset(train, test, validation)

def print_split(dataset: PartitionedDataset) -> None:
  total_len = len(dataset.train)+len(dataset.validation)+len(dataset.test)
  print(f"Train: {100*len(dataset.train)/total_len:.2f}% ({len(dataset.train)})")
  print(f"Test: {100*len(dataset.test)/total_len:.2f}% ({len(dataset.test)})")
  print(f"Validation: {100*len(dataset.validation)/total_len:.2f}% ({len(dataset.validation)})")

In [ ]:
#@title Write geotiffs.py
%%writefile geotiffs.py
"""
Helper functions for GeoTIFFs.
"""

from dataclasses import dataclass
import raster
import numpy as np

@dataclass
class Geotiffs:
  """
  Class to pass around GeoTIFFs.
  """

  def __init__(self, load_water_mask: bool = False, load_tree_mask: bool = False, regenerate_plant_nitrogen: bool = False, path: str = ""):
    self.load_water_mask = load_water_mask
    self.load_tree_mask = load_tree_mask
    self.regenerate_plant_nitrogen = regenerate_plant_nitrogen
    self.get_raster_path = lambda filename : f'{path}{filename}'

  def brazil_map_geotiff(self):
    return raster.load_raster(self.get_raster_path("brasil_clim_raster.tiff"))

  def relative_humidity_geotiff(self):
    return raster.load_raster(self.get_raster_path("R.rh_Stack.tif"))

  def temperature_geotiff(self):
    return raster.load_raster(self.get_raster_path("Temperatura_Stack.tif"))

  def vapor_pressure_deficit_geotiff(self):
    return raster.load_raster(self.get_raster_path("R.vpd_Stack.tif"))

  def atmosphere_isoscape_geotiff(self):
    return raster.load_raster(self.get_raster_path("Iso_Oxi_Stack.tif"))

  def cellulose_isoscape_geotiff(self):
    return raster.load_raster(self.get_raster_path("iso_O_cellulose.tif"))

  def soil_plant_nitrogen_difference_isoscape_geotiff(self):
    return raster.load_raster(self.get_raster_path("raster_krig_d15N_soil_plant.tiff"))

  def soil_nitrogen_isoscape_geotiff(self):
    return raster.load_raster(self.get_raster_path("raster_krig_d15N_soil.tiff"))

  def plant_nitrogen_isoscape_geotiff(self):
    return raster.load_raster(self.get_raster_path("plant_nitrogen_isoscape.tiff"))

  def carbon_means_krig_isoscape_geotiff(self):
    return raster.load_raster(self.get_raster_path("Brasil_Raster_Krig_iso_d13C.tiff"))

  def land_water_mask_geotiff(self):
    return raster.load_raster(self.get_raster_path("Land_Water_Brazil_MODIS.tif")) if self.load_water_mask else None

  def possible_tree_mask_geotiff(self):
    return raster.load_raster(self.get_raster_path("Possible_Trees_Brazil_MODIS.tif")) if self.load_tree_mask else None

  def plant_nitrogen_isoscape_geotiff(self):
    if self.regenerate_plant_nitrogen:
      plant_nitrogen_array = self.soil_nitrogen_isoscape_geotiff().yearly_masked_image - self.soil_plant_nitrogen_difference_isoscape_geotiff().yearly_masked_image
      raster.save_numpy_to_geotiff(self.soil_plant_nitrogen_difference_isoscape_geotiff(),
                                   np.expand_dims(np.flip(plant_nitrogen_array.T, axis=1), axis=2),
                                   self.get_raster_path("plant_nitrogen_isoscape.tiff"))
    return raster.load_raster(self.get_raster_path("plant_nitrogen_isoscape.tiff"))

In [ ]:
#@title Write raster.py
%%writefile raster.py
"""
Package for raster functions.
"""

from dataclasses import dataclass
import math
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal, gdal_array
from typing import List


@dataclass
class AmazonGeoTiff:
  """Represents a geotiff from our dataset."""
  gdal_dataset: gdal.Dataset
  image_value_array: np.ndarray # ndarray of floats
  image_mask_array: np.ndarray # ndarray of uint8
  masked_image: np.ma.masked_array
  yearly_masked_image: np.ma.masked_array


@dataclass
class Bounds:
  """Represents geographic bounds and size information."""
  minx: float
  maxx: float
  miny: float
  maxy: float
  pixel_size_x: float
  pixel_size_y: float
  raster_size_x: float
  raster_size_y: float

  def to_matplotlib(self) -> List[float]:
    return [self.minx, self.maxx, self.miny, self.maxy]


def print_raster_info(raster):
  dataset = raster
  print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                               dataset.GetDriver().LongName))
  print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                      dataset.RasterYSize,
                                      dataset.RasterCount))
  print("Projection is {}".format(dataset.GetProjection()))
  geotransform = dataset.GetGeoTransform()
  if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

  for band in range(dataset.RasterCount):
    band = dataset.GetRasterBand(band+1)
    #print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

    min = band.GetMinimum()
    max = band.GetMaximum()
    if not min or not max:
      (min,max) = band.ComputeRasterMinMax(False)
    #print("Min={:.3f}, Max={:.3f}".format(min,max))

    if band.GetOverviewCount() > 0:
      print("Band has {} overviews".format(band.GetOverviewCount()))

    if band.GetRasterColorTable():
      print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))


def load_raster(path: str, use_only_band_index: int = -1) -> AmazonGeoTiff:
  """
  TODO: Refactor (is_single_band, etc., should be a better design)
  --> Find a way to simplify this logic. Maybe it needs to be more abstract.
  """
  dataset = gdal.Open(path, gdal.GA_ReadOnly)
  try:
    print_raster_info(dataset)
  except AttributeError as e:
    raise OSError("Failed to print raster. This likely means it did not load properly from "+ path)
  image_datatype = dataset.GetRasterBand(1).DataType
  mask_datatype = dataset.GetRasterBand(1).GetMaskBand().DataType
  image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, 12),
                   dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))
  mask = np.zeros((dataset.RasterYSize, dataset.RasterXSize, 12),
                  dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))

  if use_only_band_index == -1:
    if dataset.RasterCount != 12 and dataset.RasterCount != 1:
      raise ValueError(f"Expected 12 raster bands (one for each month) or one annual average, but found {dataset.RasterCount}")
    if dataset.RasterCount == 1:
      use_only_band_index = 0

  is_single_band = use_only_band_index != -1

  if is_single_band and use_only_band_index >= dataset.RasterCount:
    raise IndexError(f"Specified raster band index {use_only_band_index}"
                     f" but there are only {dataset.RasterCount} rasters")

  for band_index in range(12):
    band = dataset.GetRasterBand(use_only_band_index+1 if is_single_band else band_index+1)
    image[:, :, band_index] = band.ReadAsArray()
    mask[:, :, band_index] = band.GetMaskBand().ReadAsArray()
  masked_image = np.ma.masked_where(mask == 0, image)
  yearly_masked_image = masked_image.mean(axis=2)

  return AmazonGeoTiff(dataset, image, mask, masked_image, yearly_masked_image)

def get_extent(dataset):
  geoTransform = dataset.GetGeoTransform()
  minx = geoTransform[0]
  maxy = geoTransform[3]
  maxx = minx + geoTransform[1] * dataset.RasterXSize
  miny = maxy + geoTransform[5] * dataset.RasterYSize
  return Bounds(minx, maxx, miny, maxy, geoTransform[1], geoTransform[5], dataset.RasterXSize, dataset.RasterYSize)

def plot_band(geotiff: AmazonGeoTiff, month_index, figsize=None):
  if figsize:
    plt.figure(figsize=figsize)
  im = plt.imshow(geotiff.masked_image[:,:,month_index], extent=get_extent(geotiff.gdal_dataset).to_matplotlib(), interpolation='none')
  plt.colorbar(im)

def animate(geotiff: AmazonGeoTiff, nSeconds, fps):
  fig = plt.figure( figsize=(8,8) )

  months = []
  labels = []
  for m in range(12):
    months.append(geotiff.masked_image[:,:,m])
    labels.append(f"Month: {m+1}")
  a = months[0]
  extent = get_extent(geotiff.gdal_dataset).to_matplotlib()
  ax = fig.add_subplot()
  im = fig.axes[0].imshow(a, interpolation='none', aspect='auto', extent = extent)
  txt = fig.text(0.3,0,"", fontsize=24)
  fig.colorbar(im)

  def animate_func(i):
    if i % fps == 0:
      print( '.', end ='' )

    im.set_array(months[i])
    txt.set_text(labels[i])
    return [im, txt]

  anim = animation.FuncAnimation(
      fig,
      animate_func,
      frames = nSeconds * fps,
      interval = 1000 / fps, # in ms
  )
  plt.close()

  return anim

def save_numpy_to_geotiff(bounds: Bounds, prediction: np.ma.MaskedArray, path: str):
  """Copy metadata from a base geotiff and write raster data + mask from `data`"""
  driver = gdal.GetDriverByName("GTiff")
  metadata = driver.GetMetadata()
  if metadata.get(gdal.DCAP_CREATE) != "YES":
    raise RuntimeError("GTiff driver does not support required method Create().")
  if metadata.get(gdal.DCAP_CREATECOPY) != "YES":
    raise RuntimeError("GTiff driver does not support required method CreateCopy().")

  dataset = driver.Create(path, bounds.raster_size_x, bounds.raster_size_y, prediction.shape[2], eType=gdal.GDT_Float64)
  dataset.SetGeoTransform([bounds.minx, bounds.pixel_size_x, 0, bounds.maxy, 0, bounds.pixel_size_y])
  dataset.SetProjection('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]')

  #dataset = driver.CreateCopy(path, base.gdal_dataset, strict=0)
  if len(prediction.shape) != 3 or prediction.shape[0] != bounds.raster_size_x or prediction.shape[1] != bounds.raster_size_y:
    raise ValueError("Shape of prediction does not match base geotiff")
  #if prediction.shape[2] > base.gdal_dataset.RasterCount:
  #  raise ValueError(f"Expected fewer than {dataset.RasterCount} bands in prediction but found {prediction.shape[2]}")

  prediction_transformed = np.flip(np.transpose(prediction, axes=[1,0,2]), axis=0)
  for band_index in range(dataset.RasterCount):
    band = dataset.GetRasterBand(band_index+1)
    if band.CreateMaskBand(0) == gdal.CE_Failure:
      raise RuntimeError("Failed to create mask band")
    mask_band = band.GetMaskBand()
    band.WriteArray(np.choose(prediction_transformed[:, :, band_index].mask, (prediction_transformed[:, :, band_index].data,np.array(band.GetNoDataValue()),)))
    mask_band.WriteArray(np.logical_not(prediction_transformed[:, :, band_index].mask))

def coords_to_indices(bounds: Bounds, x: float, y: float):
  if x < bounds.minx or x > bounds.maxx or y < bounds.miny or y > bounds.maxy:
    raise ValueError("Coordinates out of bounds")

  # X => lat, Y => lon
  x_idx = bounds.raster_size_y - int(math.ceil((y - bounds.miny) / abs(bounds.pixel_size_y)))
  y_idx = int((x - bounds.minx) / abs(bounds.pixel_size_x))

  return x_idx, y_idx


def get_data_at_coords(dataset: AmazonGeoTiff, x: float, y: float, month: int) -> float:
  # x = longitude
  # y = latitude
  bounds = get_extent(dataset.gdal_dataset)
  x_idx, y_idx = coords_to_indices(bounds, x, y)
  if month == -1:
    value = dataset.yearly_masked_image[x_idx, y_idx]
  else:
    value = dataset.masked_image[x_idx, y_idx, month]
  if np.ma.is_masked(value):
    raise ValueError("Coordinates are masked")
  else:
    return value

In [ ]:
%%writefile xgb_lib.py
import partition
import xgboost as xgb


def train_xgb(dataset: partition.PartitionedDataset, booster: str, rounds: int, verbose: int=0) -> xgb.XGBRegressor:
  xgb_model = xgb.XGBRegressor(n_estimators=rounds, eta=0.1, max_depth=2, objective='reg:squarederror', booster=booster)
  # split data into input and output columns
  X, y = dataset.train.iloc[:, :-1], dataset.train.iloc[:, -1]
  X_val, y_val = dataset.validation.iloc[:, :-1], dataset.validation.iloc[:, -1]
  print(f"Predicting: {dataset.train.columns[-1]}")
  xgb_model.fit(X, y, eval_set=[(X_val, y_val)], verbose=verbose)
  return xgb_model


def train_or_load_xgboost(basename: str, dataset: partition.PartitionedDataset, rounds: int=100000, verbose: int=0, rebuild: bool=False):
  if rebuild:
    print("Training model")
    model = train_xgb(dataset, booster='gblinear', rounds=rounds)
    with open(f"{basename}_config_xgb.json", "w") as f:
      f.write(model.get_booster().save_config())
    model.save_model(f"{basename}_xgb.json")
  else:
    print("Loading model")
    model = xgb.XGBRegressor()
    model.load_model(f"{basename}_xgb.json")
    with open(f"{basename}_config_xgb.json", "r") as f:
      model.get_booster().load_config(f.read())
  print(f"RMSE (validation): {model.evals_result()['validation_0']['rmse'][-1]}")
  return model

## Imports

In [ ]:
#@title Local modules
# These modules are stored in the GitHub repo at ddf-isoscapes/xgboost
import partition
import geotiffs as gts
import raster
import xgb_lib as xgb

In [ ]:
#@title Packaged modules
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
import os.path
from tqdm import tqdm
import math

rc('animation', html='jshtml')

In [ ]:
#@title Debugging
# See https://zohaib.me/debugging-in-google-collab-notebook/ for tips,
# as well as docs for pdb and ipdb.
DEBUG = False #@param {type:"boolean"}
if DEBUG:
    %pip install -Uqq ipdb
    import ipdb
    %pdb on

In [ ]:
#@title Parent Directories

# Whether to use GDrive. Currently unsupported.
USE_GDRIVE = False #@param {type:"boolean"}
# Parent directory of data if not using GDrive.
PARENT_DIR = os.path.expanduser("~/ddf/amazon_rainforest_files") #@param

# Whether to use MyDrive or a shared drive.
USE_SHARED_GDRIVE = False #@param {type:"boolean"}
# Parent directory of data in MyDrive.
PERSONAL_GDRIVE_PARENT_DIR = "amazon_rainforest_files" #@param
# Parent directory of data in shared drive. Should be prefixed with the shared drive name.
SHARED_GDRIVE_PARENT_DIR = "TNC Fellowship 🌳/4. Isotope Research & Signals/code/amazon_rainforest_files" #@param

_GDRIVE_BASE = None
if USE_GDRIVE:
  if USE_SHARED_GDRIVE:
    _GDRIVE_BASE = f"/content/drive/Shareddrives/{SHARED_GDRIVE_PARENT_DIR}"
  else:
    _GDRIVE_BASE = f"/content/drive/MyDrive/{PERSONAL_GDRIVE_PARENT_DIR}"
  _ROOT = _GDRIVE_BASE
else:
  _ROOT = PARENT_DIR


In [ ]:
#@title Data Directories
# Raster directory. Contains:
# iso_O_cellulose.tif: Isoscape of 18O from Precipitation; <-- MODELING TARGET
# Iso_Oxi_Stack.tif: Isoscape of 18O from Precipitation; <-- Model input
# R.rh_Stack.tif: Atmospheric Relative humidity <-- Model input
# R.vpd_Stack.tif: Vapor Pressure Deficit - VPD <-- Model input
# Temperature_Stack.tif: Atmospheric Temperature <-- Model input
RASTER_BASE = "amazon_rasters" #@param

SAMPLE_DATA_BASE = "amazon_sample_data" #@param
TEST_DATA_BASE = "amazon_test_data" #@param

ANIMATIONS_BASE = "amazon_animations" #@param

REBUILD_MODEL = False #@param {type:"boolean"}
MODEL_BASE = "amazon_isoscape_models" #@param

# How often should XGB log training metadata? 0 is the default, which indicates never.
XGB_VERBOSITY_LEVEL = 0 #@param

# Used to compute invalid terrain when making predictions. Leave disabled if on a low memory.
LOAD_WATER_MASK_GEOTIFF = False #@param {type:"boolean"}
LOAD_TREE_MASK_GEOTIFF = False #@param {type:"boolean"}

# If true, requires soil and plant soil nitrogen geotiffs. Also requires the following files:
# RASTER_BASE/raster_krig_d15N_soil_plant.tiff
# RASTER_BASE/raster_krig_d15N_soil.tiff
REGENERATE_PLANT_NITROGEN_GEOTIFF = False #@param {type:"boolean"}

# If false, requires XGB oxygen isoscape in MODEL_BASE/predicted_isoscape_xgboost.tiff
REGENERATE_OXYGEN_XGB_ISOSCAPE = False #@param {type:"boolean"}

# If false, requires MODEL_BASE/xgb_means_oxygen_isoscape.tiff and MODEL_BASE/xgb_variances_oxygen_isoscape.tiff
REGENERATE_OXYGEN_XGB_MEANS_VARIANCES = False #@param {type:"boolean"}

# If false, requires MODEL_BASE/plant_nitrogen_isoscape_pooled_samples.numpy.mask and MODEL_BASE/plant_nitrogen_isoscape_pooled_samples.numpy.data
REGENERATE_NITROGEN_ISOSCAPE = False #@param {type:"boolean"}

# Use Global Params to access files

In [ ]:
def get_raster_path(filename: str) -> str:
  return f"{_ROOT}/{RASTER_BASE}/{filename}"

def get_model_path(filename: str) -> str:
  return f"{_ROOT}/{MODEL_BASE}/{filename}"

def get_sample_db_path(filename: str) -> str:
  return f"{_ROOT}/{SAMPLE_DATA_BASE}/{filename}"

def get_animations_path(filename: str) -> str:
  return f"{_ROOT}/{ANIMATIONS_BASE}/{filename}"

In [ ]:
#@title Mount GDrive
# Access data stored on Google Drive
if _GDRIVE_BASE:
    from google.colab import drive
    drive.mount("/content/drive")

# Load Rasters

In [ ]:
geotiffs = gts.Geotiffs(LOAD_WATER_MASK_GEOTIFF, LOAD_TREE_MASK_GEOTIFF, REGENERATE_PLANT_NITROGEN_GEOTIFF, get_raster_path(''))

brazil_map_geotiff = geotiffs.brazil_map_geotiff() # mean annual precipitation
# Will be used to compute isoscapes for carbon and nitrogen

relative_humidity_geotiff = geotiffs.relative_humidity_geotiff()
temperature_geotiff = geotiffs.temperature_geotiff()
vapor_pressure_deficit_geotiff = geotiffs.vapor_pressure_deficit_geotiff()
atmosphere_isoscape_geotiff = geotiffs.atmosphere_isoscape_geotiff()
cellulose_isoscape_geotiff = geotiffs.cellulose_isoscape_geotiff()

# Soil Geotiffs are not necessary to load, but required to build plant nitrogen geotiff.
soil_plant_nitrogen_difference_isoscape_geotiff = geotiffs.soil_plant_nitrogen_difference_isoscape_geotiff()
soil_nitrogen_isoscape_geotiff = geotiffs.soil_nitrogen_isoscape_geotiff()
plant_nitrogen_isoscape_geotiff = geotiffs.plant_nitrogen_isoscape_geotiff()

carbon_means_krig_isoscape_geotiff = geotiffs.carbon_means_krig_isoscape_geotiff()

land_water_mask_geotiff = geotiffs.land_water_mask_geotiff()
possible_tree_mask_geotiff = geotiffs.possible_tree_mask_geotiff()

plant_nitrogen_isoscape_geotiff = geotiffs.plant_nitrogen_isoscape_geotiff()

# Train Isoscape Models

## Preprocess

Sample data from Martinelli's map of measurement sites to train fake isoscape models

In [ ]:
def gen_tabular_dataset(monthly: bool, samples_per_site: int) -> pd.DataFrame:
  sample_site_coordinates = [(-70,-5,),(-67.5,0,),(-66,-4.5,),(-63,-9.5,),(-63,-9,),(-62,-6,),(-60,-2.5,),(-60,1,),(-60,-12.5,),(-59,-2.5,),(-57.5,-4,),(-55,-3.5,),(-54,-1,),(-52.5,-13,),(-51.5,-2.5,)]
  sample_radius = 0.5
  features = [relative_humidity_geotiff, temperature_geotiff, vapor_pressure_deficit_geotiff, atmosphere_isoscape_geotiff, cellulose_isoscape_geotiff]
  image_feature_names = ["rh", "temp", "vpd", "atmosphere_oxygen_ratio", "cellulose_oxygen_ratio"]
  feature_names = ["lat", "lon", "month_of_year"] + image_feature_names
  rs = RandomState(MT19937(SeedSequence(42)))

  feature_values = {}
  for name in feature_names:
    feature_values[name] = []

  for coord in tqdm(sample_site_coordinates):
    month_start = 0 if monthly else -1
    month_end = 12 if monthly else 0
    for month in range(month_start, month_end):
      samples_collected = 0
      while samples_collected < samples_per_site:
        row = {}
        sample_x, sample_y = 2*(rs.rand(2) - 0.5) * sample_radius
        sample_x += coord[0]
        sample_y += coord[1]

        try:
          for feature, feature_name in zip(features, image_feature_names):
            row[feature_name] = raster.get_data_at_coords(feature, sample_x, sample_y, month)
          row["month_of_year"] = month
          row["lon"] = sample_x
          row["lat"] = sample_y
          samples_collected += 1
        except ValueError:
          # masked and out-of-bounds coordinates
          continue
        for key, value in row.items():
          feature_values[key].append(value)

  samples = pd.DataFrame(feature_values)

  if not monthly:
    samples.drop("month_of_year", axis=1, inplace=True)

  return samples

monthly_data_large = gen_tabular_dataset(monthly=True, samples_per_site=30)
monthly_data_255_trees = gen_tabular_dataset(monthly=True, samples_per_site=17)
yearly_data_large = gen_tabular_dataset(monthly=False, samples_per_site=30*12)
yearly_data_255_trees = gen_tabular_dataset(monthly=False, samples_per_site=17)

In [ ]:
leaf_data = pd.read_csv(get_sample_db_path("pontos-vasp-cluster.csv"))
leaf_data.head()

In [ ]:
def load_leaf_dataframe(db_path: str, isotope_col: str):
  leaf_data = pd.read_csv(db_path)
  leaf_data = leaf_data.rename(columns={"latitude": "lat", "longitude": "lon"})
  leaf_df = leaf_data[["lon", "lat", "MAP", "MAT", "vap", "d15N_soil", "dem", "pa", "pet", "ph", isotope_col]]
  return leaf_df

carbon_df = load_leaf_dataframe(get_sample_db_path("pontos-vasp-cluster.csv"), "d13C")
nitrogen_df = load_leaf_dataframe(get_sample_db_path("pontos-vasp-cluster.csv"), "d15N")

### Partition

In [ ]:
yearly_large_partitioned = data.partition(yearly_data_large)
data.print_split(yearly_large_partitioned)

In [ ]:
yearly_255_trees_partitioned = data.partition(yearly_data_255_trees)
data.print_split(yearly_255_trees_partitioned)

In [ ]:
monthly_large_partitioned = data.partition(monthly_data_large)
data.print_split(monthly_large_partitioned)

In [ ]:
monthly_255_trees_partitioned = data.partition(monthly_data_255_trees)
data.print_split(monthly_255_trees_partitioned)

In [ ]:
nitrogen_df_partitioned = data.partition(nitrogen_df)
data.print_split(nitrogen_df_partitioned)

In [ ]:
carbon_df_partitioned = data.partition(carbon_df)
data.print_split(carbon_df_partitioned)

## XGBoost: Train XGBoost Models

In [ ]:
# Validation RMSE xgboost: 0.306059 w/ 100,000 rounds
# Validation RMSE google internal tooling: 0.39386
yearly_255_trees_xgb_model = xgb.train_or_load_xgboost(
  get_model_path("oxygen_isoscape_model"),
  yearly_255_trees_partitioned,
  rounds=100000,
  verbose=XGB_VERBOSITY_LEVEL,
  rebuild=REBUILD_MODEL
)

In [ ]:
# HMM, post-bugfix, Carbon might diverge too.
carbon_isoscape_model = xgb.train_or_load_xgboost(get_model_path("carbon_isoscape_model"), carbon_df_partitioned, verbose=XGB_VERBOSITY_LEVEL, rebuild=REBUILD_MODEL)

In [ ]:
# Validation loss seems to diverge
nitrogen_isoscape_model = xgb.train_or_load_xgboost(get_model_path("nitrogen_isoscape_model"), nitrogen_df_partitioned, rounds=10000, verbose=XGB_VERBOSITY_LEVEL, rebuild=REBUILD_MODEL)

### Test XGBoost Model Code

**In addition to unit tests, we also trained a model assuming 255 trees sampled monthly.**

Preserving this as text only because it is not realistic as of 2023.

Validation RMSE xgboost: 0.29072 \
Validation RMSE Google internal tooling: 0.29183 \
`monthly_255_trees_xgb_model = train_xgb(monthly_255_trees_partitioned, booster='gbtree', rounds=15000, XGB_VERBOSITY_LEVEL)`

For the best results here, add `max_depth=2` to XGBRegressor params.

# Compute Isoscapes

## XGBoost: Compute AI-Predicted Isoscape

Required: REGENERATE_OXYGEN_XGB_ISOSCAPE == true

In [ ]:
def get_xgb_isoscape_prediction():
  bounds = raster.get_extent(cellulose_isoscape_geotiff.gdal_dataset)
  features = [relative_humidity_geotiff, temperature_geotiff, vapor_pressure_deficit_geotiff, atmosphere_isoscape_geotiff]
  image_feature_names = ["rh", "temp", "vpd", "atmosphere_oxygen_ratio"]
  #feature_names = ["lat", "lon", "month_of_year"] + image_feature_names
  feature_names = ["lat", "lon"] + image_feature_names
  predicted_isoscape = np.ma.array(np.zeros([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=float), mask=np.ones([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=bool))

  for x_idx, x in enumerate(tqdm(np.arange(bounds.minx, bounds.maxx, bounds.pixel_size_x, dtype=float))):
    rows = []
    row_indexes = []
    for y_idx, y in enumerate(np.arange(bounds.miny, bounds.maxy, -bounds.pixel_size_y, dtype=float)):
      #for month in range(12):
      month = 0
      row = {}
      try:
        for feature, feature_name in zip(features, image_feature_names):
          row[feature_name] = raster.get_data_at_coords(feature, x, y, month)
        #row["month_of_year"] = month
        row["lon"] = x
        row["lat"] = y
      except ValueError:
        # masked and out-of-bounds coordinates
        continue
      except IndexError:
        continue
      rows.append(row)
      row_indexes.append((y_idx,month,))
    if (len(rows) > 0):
      reordered = pd.DataFrame(rows)[yearly_255_trees_xgb_model.get_booster().feature_names]
      predictions = yearly_255_trees_xgb_model.predict(reordered)
      predictions_np = predictions
      for prediction, (y_idx, month_idx) in zip(predictions_np, row_indexes):
        predicted_isoscape.mask[x_idx,y_idx,month_idx] = False # unmask since we have data
        predicted_isoscape.data[x_idx,y_idx,month_idx] = prediction

  return predicted_isoscape

if REGENERATE_OXYGEN_XGB_ISOSCAPE:
  xgb_isoscape_prediction = get_xgb_isoscape_prediction()
  raster.save_numpy_to_geotiff(raster.get_extent(cellulose_isoscape_geotiff.gdal_dataset), xgb_isoscape_prediction, get_model_path("predicted_isoscape_xgboost.tiff"))
  plt.imshow(xgb_isoscape_prediction)

# TODO: TESTME!

## Turn XGBoost isoscape into a Gaussian distribution

In [ ]:
predicted_cellulose_isoscape_geotiff = raster.load_raster(get_model_path("predicted_isoscape_xgboost.tiff"))

In [ ]:
plt.imshow(predicted_cellulose_isoscape_geotiff.yearly_masked_image)

In [ ]:
from scipy.stats import multivariate_normal

def get_2d_gaussian(center_lon: float, center_lat: float, stdev: float):
  """Quick-and-dirty function to get a PDF for sampling from an image
  to turn it into a distribution. Intended for use with isoscapes.

  Major room for improvement! This framing assumes no distortion from the
  projection, i.e. that 1 deg latitude == 1 deg longitude == 111 km everywhere.
  This should probably be fine for Brazil, for now, since it's near the Equator.
  """
  rv = multivariate_normal([center_lat, center_lon], [[stdev, 0], [0, stdev]])

  return rv

# x = longitude
# y = latitude
def plot_gaussian(rv, shape: raster.Bounds):
  """Informative, for debugging and visualizing get_2d_gaussian()."""
  x = np.linspace(shape.minx, shape.maxx, shape.raster_size_x)
  y = np.linspace(shape.maxy, shape.miny, shape.raster_size_y) # inverted y axis
  X, Y = np.meshgrid(x,y)
  target = np.empty((shape.raster_size_y, shape.raster_size_x,2,), dtype=float)
  target[:, :, 0] = Y
  target[:, :, 1] = X
  pd = rv.pdf(target)
  plt.imshow(pd)
  plt.colorbar()

# TODO: For each pixel in the predicted isoscape
# Yeah, this is basically Gaussian blur, huh...
# BUT, it does give us a distribution.
def gaussian_kernel(input: raster.AmazonGeoTiff, stdev_in_degrees: float=1):
  bounds = raster.get_extent(input.gdal_dataset)
  means = np.ma.zeros((bounds.raster_size_x, bounds.raster_size_y,), dtype=float)
  means.mask = np.ones((bounds.raster_size_x, bounds.raster_size_y), dtype=bool)
  variances = np.ma.zeros((bounds.raster_size_x, bounds.raster_size_y,), dtype=float)
  variances.mask = np.ones((bounds.raster_size_x, bounds.raster_size_y), dtype=bool)
  for map_y in tqdm(range(0, bounds.raster_size_y, 1)):
    y_coord = map_y * abs(bounds.pixel_size_y) + bounds.miny
    for map_x in range(0, bounds.raster_size_x, 1):
      x_coord = map_x * abs(bounds.pixel_size_x) + bounds.minx
      rv = get_2d_gaussian(y_coord, x_coord, stdev_in_degrees)
      rsamp = rv.rvs(1000)
      values = []
      for coordinate_pair in rsamp:
        try:
          #print(coordinate_pair)
          values.append(raster.get_data_at_coords(input, coordinate_pair[0], coordinate_pair[1], 0))
        except ValueError:
          pass
        if len(values) == 30:
          break
      if len(values) == 30:
        # Set the mean and stdev pixels
        #print(x_coord, y_coord)
        means[map_x, map_y] = np.mean(values)
        variances[map_x, map_y] = np.var(values)
        # Apply sample corrective factor to variance
        variances[map_x, map_y] *= len(values) / (len(values)-1)
        means.mask[map_x, map_y] = False
        variances.mask[map_x, map_y] = False
  return means, variances

rv = get_2d_gaussian(-60.16, 4.11, 1)
plot_gaussian(rv, raster.get_extent(predicted_cellulose_isoscape_geotiff.gdal_dataset))

if REGENERATE_OXYGEN_XGB_MEANS_VARIANCES:
  xgb_means, xgb_variances = gaussian_kernel(predicted_cellulose_isoscape_geotiff, stdev_in_degrees=0.1)
  bds = raster.get_extent(predicted_cellulose_isoscape_geotiff.gdal_dataset)
  raster.save_numpy_to_geotiff(bds, np.expand_dims(xgb_means, axis=2), get_model_path("xgb_means_oxygen_isoscape.tiff"))
  raster.save_numpy_to_geotiff(bds, np.expand_dims(xgb_variances, axis=2), get_model_path("xgb_variances_oxygen_isoscape.tiff"))

In [ ]:
xgb_means_oxygen_geotiff = raster.load_raster(get_model_path("xgb_means_oxygen_isoscape.tiff"))
xgb_variances_oxygen_geotiff = raster.load_raster(get_model_path("xgb_variances_oxygen_isoscape.tiff"))

# Until we re-generate the map
xgb_variances_oxygen_geotiff.yearly_masked_image *= (5 / 4)
xgb_variances_oxygen_geotiff.masked_image *= (5 / 4)

In [ ]:
raster.plot_band(xgb_means_oxygen_geotiff, 0)

In [ ]:
raster.plot_band(xgb_variances_oxygen_geotiff, 0)

In [ ]:
raster.get_data_at_coords(xgb_means_oxygen_geotiff, -60, 4, 0)

# Evaluate Precision, Recall From Samples

## Oxygen

In [ ]:
real_samples = pd.read_csv(get_sample_db_path("38_ISOTOPE RESULTS _VARIABLES_10032023.csv"), sep=';')

In [ ]:
import scipy.stats
import random
g = real_samples.groupby(['lat','long'])['d18O sample']
print('lat,long,mean,var,n,p_value,reject?')
print("Real Coordinates, Real Samples")
all_coords = []
for coords, x in g:
  if x.size > 1:
    lat = coords[0]
    lon = coords[1]
    all_coords.append(coords)
    lab_samp_mean = x.mean()
    lab_samp_var = x.var()*(x.size / (x.size - 1))
    lab_samp_size = x.size
    sumauma_samp_mean = raster.get_data_at_coords(xgb_means_oxygen_geotiff, lon, lat, 0)
    sumauma_samp_var = raster.get_data_at_coords(xgb_variances_oxygen_geotiff, lon, lat, 0)
    sumauma_samp_size = 5
    _, p_value = scipy.stats.ttest_ind_from_stats(sumauma_samp_mean, math.sqrt(sumauma_samp_var), sumauma_samp_size, lab_samp_mean, math.sqrt(lab_samp_var), lab_samp_size, equal_var=False, alternative="two-sided")
    print(f"{lat},{lon},{lab_samp_mean},{lab_samp_var},{lab_samp_size},{p_value},{p_value < 0.05}")

print("Fake Coordinates, Real Samples")
for _, x in g:
  if x.size > 1:
    coords = random.choice(all_coords)
    lat = coords[0]
    lon = coords[1]
    lab_samp_mean = x.mean()
    lab_samp_var = x.var()*(x.size / (x.size - 1))
    lab_samp_size = x.size
    sumauma_samp_mean = raster.get_data_at_coords(xgb_means_oxygen_geotiff, lon, lat, 0)
    sumauma_samp_var = raster.get_data_at_coords(xgb_variances_oxygen_geotiff, lon, lat, 0)
    sumauma_samp_size = 5
    _, p_value = scipy.stats.ttest_ind_from_stats(sumauma_samp_mean, math.sqrt(sumauma_samp_var), sumauma_samp_size, lab_samp_mean, math.sqrt(lab_samp_var), lab_samp_size, equal_var=False, alternative="two-sided")
    print(f"{lat},{lon},{lab_samp_mean},{lab_samp_var},{lab_samp_size},{p_value},{p_value < 0.05}")


## Carbon

In [ ]:
real_samples_old = pd.read_csv(get_sample_db_path("pontos-vasp-cluster.csv"), sep=',')

In [ ]:
carbon_means_geotiff = raster.load_raster(get_raster_path("iso_d13C_map_wood_stack.tiff"), use_only_band_index=0)

In [ ]:
carbon_variances_geotiff = raster.load_raster(get_raster_path("iso_d13C_map_wood_stack.tiff"), use_only_band_index=1)

In [ ]:
import scipy.stats
g = real_samples_old.groupby(['latitude','longitude'])[['d13C', 'd15N']]
# Assume sample variance == Kriging variance b/c we only have means for the samples in this CSV
print('lat,long,mean,var,n,p_value,reject?')
print("Real Coordinates, Real Samples")
all_coords = []
for coords, x in g:
  try:
    if x.size >= 1:
      lat = coords[0]
      lon = coords[1]
      all_coords.append(coords)
      lab_samp_size = 5
      sumauma_samp_size = 5 # for all we know

      # d13C p-value
      lab_samp_mean_d13c = x['d13C'].mean()
      lab_samp_var_d13c = raster.get_data_at_coords(carbon_variances_geotiff, lon, lat, 0)
      sumauma_samp_mean_d13c = raster.get_data_at_coords(carbon_means_geotiff, lon, lat, 0)
      sumauma_samp_var_d13c = raster.get_data_at_coords(carbon_variances_geotiff, lon, lat, 0)
      _, p_value_d13c = scipy.stats.ttest_ind_from_stats(sumauma_samp_mean_d13c, math.sqrt(sumauma_samp_var_d13c), sumauma_samp_size, lab_samp_mean_d13c, math.sqrt(lab_samp_var_d13c), lab_samp_size, equal_var=True, alternative="two-sided")

      # d15N p-value
      if False:
        # Waiting for real d15N rasters
        lab_samp_mean_d15n = x['d15N'].mean()
        lab_samp_var_d15n = raster.get_data_at_coords(nitrogen_variances_geotiff, lon, lat, 0)
        sumauma_samp_mean_d15n = raster.get_data_at_coords(nitrogen_means_geotiff, lon, lat, 0)
        sumauma_samp_var_d15n = raster.get_data_at_coords(nitrogen_variances_geotiff, lon, lat, 0)
        _, p_value_d15n = scipy.stats.ttest_ind_from_stats(sumauma_samp_mean_d15n, math.sqrt(sumauma_samp_var_d15n), sumauma_samp_size, lab_samp_mean_d15n, math.sqrt(lab_samp_var_d15n), lab_samp_size, equal_var=True, alternative="two-sided")
      print(f"{lat},{lon},{lab_samp_mean_d13c},{lab_samp_var_d13c},{lab_samp_size},{p_value_d13c},{p_value_d13c < 0.05}")
  except ValueError:
    pass

print("Fake Coordinates, Real Samples")
for real_coords, x in g:
  try:
    if x.size >= 1:
      coords = random.choice(all_coords)
      while coords[0] == real_coords[0] and coords[1] == real_coords[1]:
        coords = random.choice(all_coords)
      lat = coords[0]
      lon = coords[1]
      lab_samp_mean = x['d13C'].mean()
      lab_samp_var = raster.get_data_at_coords(carbon_variances_geotiff, lon, lat, 0)
      lab_samp_size = x.size
      sumauma_samp_mean = raster.get_data_at_coords(carbon_means_geotiff, lon, lat, 0)
      sumauma_samp_var = raster.get_data_at_coords(carbon_variances_geotiff, lon, lat, 0)
      sumauma_samp_size = 5
      _, p_value = scipy.stats.ttest_ind_from_stats(sumauma_samp_mean, math.sqrt(sumauma_samp_var), sumauma_samp_size, lab_samp_mean, math.sqrt(lab_samp_var), lab_samp_size, equal_var=True, alternative="two-sided")
      print(f"{lat},{lon},{lab_samp_mean},{lab_samp_var},{lab_samp_size},{p_value},{p_value < 0.05}")
  except ValueError:
    pass


# A probability that a coordinate matches a sample given a predicted isoscape

1. Compute t-intervals for a coordinate pixel on the paperwork based on the output of the AI isoscape model
2. For a given sample, compute a z-score from the estimated t-distribution, and turn that into a p-value based on two-sided area under the curve: p($\in$ isotope distribution | possible coordinates, isoscape). *This can be combined with other knowledge in the future to get, for example, p($\in$ isoscape distribution $\land$ tree rings look right for the area | possible coordinates, isoscape, tree ring knowledge).*
3. Depending on the p-value, reject the null hypothesis that the paperwork is correct

Key challenge with this approach: The statistical bound on false positives is per-pixel, and only guarantees that it falls in a range of isoscape values associated with that location rather than the location itself. We will likely have multiple possible points of origin for each sample.

## Compute Sample Origin Given AI-Predicted Isoscapes

In [ ]:
predicted_cellulose_isoscape_geotiff = raster.load_raster(get_model_path("predicted_isoscape_xgboost.tiff"))

In [ ]:
try:
    np.sum(predicted_cellulose_isoscape_geotiff.masked_image.mask[0,:,:])
except IndexError as err:
    raise IndexError(err + " If you're seeing this error, the image mask is unexpectedly missing. You might need to rerun the Oxygen xgboost trainer.")


In [ ]:
raster.plot_band(predicted_cellulose_isoscape_geotiff, 1, figsize=(12,12))

In [ ]:
plt.imshow(predicted_cellulose_isoscape_geotiff.masked_image.data[:,:,3])

### Compute p-values for a sample
Null hypotheses in these tests always involve equality. Our null hypothesis is that the sample could reasonably come from any given location. We reject that null hypothesis when it is very improbably that the sample came from a given location (i.e. p < c).

Should be a two-sample test: One sample from the Craig-Gordon or ML approximation, one sample from the tree cellulose, ideally with N >= 30 for each.

\
H0 (null hypothesis): pixel group == sample; i.e. the sample could be from x \
Ha (alternative hypothesis): pixel group != sample; i.e. the sample might not be from x

\
Many coordinates work well like (-55,-5), but some-- especially those affected by the saturation-- don't, like (-55,-10). We would incorrectly rule those areas out as potential origins of a wood sample by rejecting the null hypothesis at reasonable p-values like 0.05. We could require substantially lower p-values to reject, but that would come at the cost of ruling very little out (i.e. not the most useful model).
**TODO: To remedy this, consider using a larger sample size (> 1 pixel) for the isoscape side of the t-test**

#### Code

In [ ]:
def make_isoscape_with_pooled_sample_dimension_from_2d_isoscape(input: np.ma.MaskedArray, radius: int = 5):

  canvas = np.zeros((radius*3, radius*3), dtype=float)
  x_origin = int((radius*3)/2)
  y_origin = int((radius*3)/2)
  for y in range(-radius, radius):
    for x in range(-radius, radius):
      if (x*x+y*y <= radius*radius - radius):
        x_pixel = min(max(x+x_origin, 0), radius*3-1)
        y_pixel = min(max(y+y_origin, 0), radius*3-1)
        canvas[x_pixel, y_pixel] = 1
  area = int(sum(canvas.flatten()))

  pooled = np.ma.MaskedArray(data=np.zeros((input.shape[0], input.shape[1], area), dtype=float),
                             mask=np.repeat(input.mask[:, :, np.newaxis], area, axis=2))


  for x_origin in tqdm(range(input.shape[0])):
    for y_origin in range(input.shape[1]):
      pooling_counter = 0
      for y in range(-radius, radius):
        for x in range(-radius, radius):
          if (x*x+y*y <= radius*radius - radius):
            x_pixel = x+x_origin
            y_pixel = y+y_origin
            x_pixel_clamped = min(max(x_pixel, 0), input.shape[0]-1)
            y_pixel_clamped = min(max(y_pixel, 0), input.shape[1]-1)
            if x_pixel_clamped == x_pixel and y_pixel_clamped == y_pixel:
              pooled[x_origin, y_origin, pooling_counter] = input[x_pixel, y_pixel]
              pooling_counter += 1
      if pooling_counter < area:
        samples_to_mask = area - pooling_counter
        pooled.mask[x_origin, y_origin, pooling_counter:] = True
  return pooled

In [ ]:
if REGENERATE_NITROGEN_ISOSCAPE:
    plant_nitrogen_isoscape_pooled_samples = make_isoscape_with_pooled_sample_dimension_from_2d_isoscape(plant_nitrogen_isoscape_geotiff.yearly_masked_image, 5)
    np.save(get_model_path("plant_nitrogen_isoscape_pooled_samples.numpy.mask"), plant_nitrogen_isoscape_pooled_samples.mask)
    np.save(get_model_path("plant_nitrogen_isoscape_pooled_samples.numpy.data"), plant_nitrogen_isoscape_pooled_samples.data)

In [ ]:
mask = np.load(get_model_path("plant_nitrogen_isoscape_pooled_samples.numpy.mask.npy"))
data = np.load(get_model_path("plant_nitrogen_isoscape_pooled_samples.numpy.data.npy"))
plant_nitrogen_isoscape_pooled_samples = np.ma.MaskedArray(data, mask=mask)

In [ ]:
import dataclasses
import scipy.stats
import cv2

def sample_from_geotiff(x, y, sample_geotiff, sample_radius, num_samples):
  """
  Take a sample from the "real" data.
  In practice, this would be multiple wood samples from the same furniture
  Here, these come from slightly different pixels in a geotiff.
  """

  coord = (x, y,)
  rs = RandomState(MT19937(SeedSequence(42)))
  samples = []

  for _ in range(num_samples):
    sample_x, sample_y = 2*(rs.rand(2) - 0.5) * sample_radius
    sample_x += coord[0]
    sample_y += coord[1]
    total_months = sample_geotiff.masked_image.shape[2]
    monthly_readings = [raster.get_data_at_coords(sample_geotiff, sample_x, sample_y, month) for month in range(total_months)]
    samples.append(np.mean(monthly_readings))
  return samples

def fake_t_test(isoscape_geotiff_masked_image, cellulose_sample_x, cellulose_sample_y, cellulose_samples_geotiff, num_cellulose_samples):
  """Called a "fake" t-test because we randomly sample points close to
     (cellulose_sample_x, cellulose_sample_y) from `cellulose_samples_geotiff`
     to mimic taking samples out of the same piece of furniture.

     We then perform a series of two-sample t-tests between this fake
     furniture/timber sample and each coordinate of isoscape_geotiff
  """
  fake_sample = sample_from_geotiff(cellulose_sample_x, cellulose_sample_y, cellulose_samples_geotiff, raster.get_extent(cellulose_samples_geotiff.gdal_dataset).pixel_size_x*50, num_cellulose_samples)
  shape = isoscape_geotiff_masked_image.shape
  return scipy.stats.ttest_ind(isoscape_geotiff_masked_image, np.tile(fake_sample,(shape[0],shape[1],1)), axis=2, equal_var=False, alternative='two-sided')

def crop_to_coordinates(original_extent: raster.Bounds, new_extent: raster.Bounds, original_image: np.ndarray):
  x_off_deg = max((new_extent.minx - original_extent.minx), 0)
  x_off_px = int(x_off_deg / abs(original_extent.pixel_size_x))
  y_off_deg = max((new_extent.miny - original_extent.miny), 0)
  y_off_px = int(y_off_deg / abs(original_extent.pixel_size_y))
  x_max_deg = min(new_extent.maxx, original_extent.maxx)
  x_max_px = int((x_max_deg - original_extent.minx) / abs(original_extent.pixel_size_x))
  y_max_deg = min(new_extent.maxy, original_extent.maxy)
  y_max_px = int((y_max_deg - original_extent.miny) / abs(original_extent.pixel_size_y))
  # Pixels are stored in descending order if an axis has negative pixel size.
  if original_extent.pixel_size_x < 0:
    raise RuntimeError("Inverted X axis not supported")
  cropped_image = np.flip(original_image, axis=0) if original_extent.pixel_size_y < 0 else original_image
  cropped_image = cropped_image[y_off_px:y_max_px, x_off_px:x_max_px]
  cropped_image = np.flip(cropped_image, axis=0) if original_extent.pixel_size_y < 0 else cropped_image
  resulting_bounds = raster.Bounds(original_extent.minx + x_off_deg,
                            x_max_deg,
                            original_extent.miny + y_off_deg,
                            y_max_deg,
                            original_extent.pixel_size_x,
                            original_extent.pixel_size_y,
                            cropped_image.shape[1],
                            cropped_image.shape[0])
  return cropped_image, resulting_bounds

def pad_to_coordinates(original_extent: raster.Bounds, new_extent: raster.Bounds, original_image: np.ndarray, with_ones: bool = False):
  """Pads an image to new coordinates larger than the original.
  Precondition: Must not specify negative padding (i.e. a crop)
  """
  x_size = max(abs(int((new_extent.maxx - new_extent.minx) / original_extent.pixel_size_x)), original_image.T.shape[0])
  y_size = max(abs(int((new_extent.maxy - new_extent.miny) / original_extent.pixel_size_y)), original_image.T.shape[1])
  if with_ones:
    padded_image = np.ones((x_size, y_size,), dtype=original_image.dtype)
  else:
    padded_image = np.zeros((x_size, y_size,), dtype=original_image.dtype)
  x_offset = max(int((original_extent.minx - new_extent.minx) / abs(original_extent.pixel_size_x)), 0)
  y_offset = max(int((original_extent.miny - new_extent.miny) / abs(original_extent.pixel_size_y)), 0)
  # Correct rounding errors to satisfy the following by adjusting offsets:
  # * x_offset + original_extent.raster_size_x <= padded_image.shape[0]
  # --> x_offset <= padded_image.shape[0] - original_extent.raster_size_x
  # * y_offset + original_extent.raster_size_y <= padded_image.shape[1]
  x_offset = min(x_offset, padded_image.shape[0] - original_image.T.shape[0])
  y_offset = min(y_offset, padded_image.shape[1] - original_image.T.shape[1])

  # Pixels are stored in descending order if an axis has negative pixel size.
  if original_extent.pixel_size_x < 0:
    raise RuntimeError("Inverted X axis not supported")
  padded_image = np.flip(padded_image, axis=1) if original_extent.pixel_size_y < 0 else padded_image
  original_image = np.flip(original_image, axis=0) if original_extent.pixel_size_y < 0 else original_image
  padded_image[x_offset:x_offset+original_image.T.shape[0], y_offset:y_offset+original_image.T.shape[1]] = original_image.T
  padded_image = np.flip(padded_image, axis=1) if original_extent.pixel_size_y < 0 else padded_image
  return padded_image.T

def upscale(new_size_x: int, new_size_y: int, original_image: np.ndarray):
  """Rescales an image of the same geographical extent with nearest-neighbor sampling.

  Useful to match pixel size between images.
  """
  resized_data = cv2.resize(original_image, dsize=(new_size_y, new_size_x), interpolation=cv2.INTER_NEAREST_EXACT)
  return resized_data

def align_to_bounds(original_bounds: raster.Bounds, new_bounds: raster.Bounds, original_image: np.ndarray, pad_with_ones: bool):
  # TODO: UNIT TESTS (if we want to use this version in production)!!!
  cropped, cropped_bounds = crop_to_coordinates(original_bounds, new_bounds, original_image)
  padded = pad_to_coordinates(cropped_bounds, new_bounds, cropped, with_ones=pad_with_ones)
  scaled = upscale(new_bounds.raster_size_x, new_bounds.raster_size_y, padded)
  return scaled

def combine_pvalues(pvalue_nitrogen, pvalue_oxygen, nitrogen_extent, oxygen_extent, nitrogen_mask, oxygen_mask):
  # TODO: implement combine_pvalues() in a generic way to combine any two maskedarrays of p-values
  # using align_to_bounds()
  pvalue_nitrogen = pad_to_coordinates(nitrogen_extent, oxygen_extent, pvalue_nitrogen)
  nitrogen_mask = pad_to_coordinates(nitrogen_extent, oxygen_extent, nitrogen_mask.astype(int), True).astype(bool)
  pvalue_oxygen = upscale(pvalue_nitrogen.shape[0], pvalue_nitrogen.shape[1], pvalue_oxygen)
  oxygen_mask = upscale(pvalue_nitrogen.shape[0], pvalue_nitrogen.shape[1], oxygen_mask.astype(int)).astype(bool)
  both_mask = np.logical_or(oxygen_mask, nitrogen_mask)
  pvalues_combined = np.ma.MaskedArray(pvalue_oxygen*pvalue_nitrogen, mask=both_mask)
  result = np.ma.masked_array(np.where(both_mask, pvalue_oxygen, pvalues_combined), mask=oxygen_mask)
  return result

def evalutate_on_sample_from_point(x, y, p_threshold=0.05, num_cellulose_samples=10):
  print(predicted_cellulose_isoscape_geotiff.masked_image.mask.shape)
  default_mask = predicted_cellulose_isoscape_geotiff.masked_image.mask[:,:,0]
  oxygen_extent=raster.get_extent(predicted_cellulose_isoscape_geotiff.gdal_dataset)
  nitrogen_extent=raster.get_extent(plant_nitrogen_isoscape_geotiff.gdal_dataset)
  nitrogen_mask = plant_nitrogen_isoscape_geotiff.yearly_masked_image.mask
  oxygen_mask = predicted_cellulose_isoscape_geotiff.yearly_masked_image.mask
  # Pessimistic case: Cellulose sample is faked with Craig-Gordon model, and
  # the isoscape comes from the AI model. Nitrogen comes from Martinelli's
  # geotiff (unknown ultimate source).
  statistic_oxygen, pvalue_oxygen = fake_t_test(predicted_cellulose_isoscape_geotiff.masked_image, x, y, cellulose_isoscape_geotiff, num_cellulose_samples)
  statistic_nitrogen, pvalue_nitrogen = fake_t_test(plant_nitrogen_isoscape_pooled_samples, x, y, plant_nitrogen_isoscape_geotiff, num_cellulose_samples)
  pvalues_combined = combine_pvalues(pvalue_nitrogen, pvalue_oxygen, nitrogen_extent, oxygen_extent, nitrogen_mask, oxygen_mask)

  pvalues_bounds = dataclasses.replace(oxygen_extent,
                                       raster_size_x = pvalues_combined.shape[0],
                                       raster_size_y = pvalues_combined.shape[1],
                                       pixel_size_x = nitrogen_extent.pixel_size_x,
                                       pixel_size_y = nitrogen_extent.pixel_size_y)
  land_water_mask = align_to_bounds(raster.get_extent(land_water_mask_geotiff.gdal_dataset), pvalues_bounds, land_water_mask_geotiff.masked_image[:,:,0], pad_with_ones=True)
  possible_tree_mask = align_to_bounds(raster.get_extent(possible_tree_mask_geotiff.gdal_dataset), pvalues_bounds, possible_tree_mask_geotiff.masked_image[:,:,0], pad_with_ones=True)

  fig, ax = plt.subplots(2,2, figsize=(30,30))
  plt.colorbar(ax[0,0].imshow(pvalues_combined, extent=oxygen_extent.to_matplotlib()), ax=ax[0,0])
  ax[0,0].set_title("p-values")
  invalid_terrain = np.logical_or(np.logical_or(pvalues_combined.mask, land_water_mask), np.logical_not(possible_tree_mask))
  ax[0,1].imshow(np.ma.masked_array(pvalues_combined < p_threshold, mask=invalid_terrain), extent=oxygen_extent.to_matplotlib())
  circle1 = plt.Circle((x, y), 1, color='r')
  ax[0,1].add_patch(circle1)
  ax[0,1].set_title(f"p < {p_threshold}")
  ax[0,0].set_ylabel("Pessimistic Case")

  print(f"Invalid p-values: {np.sum(pvalues_combined.flatten() > 1)}")
  # Optimistic case: Cellulose sample is faked with Craig-Gordon model, and
  # the isoscape also comes from the Craig-Gordon model.
  statistic_oxygen, pvalue_oxygen = fake_t_test(cellulose_isoscape_geotiff.masked_image, x, y, cellulose_isoscape_geotiff, num_cellulose_samples)
  pvalues_combined = combine_pvalues(pvalue_nitrogen, pvalue_oxygen, nitrogen_extent, oxygen_extent, nitrogen_mask, oxygen_mask)
  oxygen_extent=raster.get_extent(cellulose_isoscape_geotiff.gdal_dataset)
  plt.colorbar(ax[1,0].imshow(pvalues_combined, extent=oxygen_extent.to_matplotlib()), ax=ax[1,0])
  #ax[1,0].imshow(possible_tree_mask, extent=oxygen_extent.to_matplotlib())
  #ax[1,0].imshow(invalid_terrain.astype(bool), extent=oxygen_extent.to_matplotlib())
  ax[1,1].imshow(np.ma.masked_array(pvalues_combined < p_threshold, invalid_terrain), extent=oxygen_extent.to_matplotlib())
  circle1 = plt.Circle((x, y), 1, color='r')
  ax[1,1].add_patch(circle1)
  ax[1,0].set_ylabel("Optimistic Case")

#### Story

**Yellow: We reject the null hypothesis that the sample might have come from here because it is so improbable**

##### Pessimistic Example: Outlier point sampled from outlier area

In [ ]:
evalutate_on_sample_from_point(x=-55, y=-10, p_threshold=0.01, num_cellulose_samples=10)

##### More Optimistic Cases

In [ ]:
evalutate_on_sample_from_point(x=-55, y=-5, p_threshold=0.01, num_cellulose_samples=10)

In [ ]:
evalutate_on_sample_from_point(x=-65, y=-5, p_threshold=0.01, num_cellulose_samples=10)

In [ ]:
# This is the Amazon biome shapefile we use:
# https://services.arcgis.com/F7DSX1DSNSiWmOqh/arcgis/rest/services/lm_bioma_250/FeatureServer
# http://geoftp.ibge.gov.br/informacoes_ambientais/estudos_ambientais/biomas/vetores/Biomas_250mil.zip

# TODO: Figure out how to produce good isoscapes

### Compute mean absolute error in km
*Compute based on min, max, and median AE for each sample.*
Also compute proportion of samples for which nothing passed our threshold.
Ideally, plot these against each other on a curve while varying $c$ (0.95, 0.99, etc.).

This may not be what we actually want because ultimately this is a binary classification problem. That framing may be more useful as a business metric.